In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime ,time

In [2]:
# rumor source tweets
src = pd.read_csv(r"D:\論文\PHEME9\Code\sydneysiege\18 targetanalysis\targetdata\sydneysiege-target-src.csv", encoding = 'utf-8', header = 0)

In [3]:
# src.dropna(inplace=True)
src.isnull().sum()

src_tweet_id      0
src_user_id       0
src_date          0
rumorlabel        0
src_tweet         0
cleaned_src_tw    0
targetlabel       0
src_sentiment     0
dtype: int64

In [4]:
src.targetlabel.value_counts()

0    415
1     84
Name: targetlabel, dtype: int64

In [5]:
src.src_date = pd.to_datetime(src.src_date)

In [6]:
src.sort_values(by='src_date',inplace=True)
src.reset_index(inplace=True,drop=True)
src.head()

,src_tweet_id,src_user_id,src_date,rumorlabel,src_tweet,cleaned_src_tw,targetlabel,src_sentiment
0,544267656597995521,258627226,2014-12-14 23:08:15+00:00,1,BREAKING: Hostages are being forced to hold an...,break hostages force hold isis flag lindt cafe...,1,Neutral
1,544268186292867072,1960878613,2014-12-14 23:10:22+00:00,1,BREAKING: Gunman takes hostages in cafe in Mar...,break gunman take hostages cafe martin place s...,1,Neutral
2,544268637134393344,171802941,2014-12-14 23:12:09+00:00,1,UPDATE: An ISIS flag is being displayed in the...,update isis flag display window café siege syd...,1,Neutral
3,544268732046913536,16834659,2014-12-14 23:12:32+00:00,1,#BREAKING: Hostages are being held and a siege...,break hostages hold siege take place sydneys l...,0,Neutral
4,544269221564137472,258627226,2014-12-14 23:14:29+00:00,1,BREAKING: A Sydney cafe at Martin Place is bei...,break sydney cafe martin place hold hostages i...,1,Positive


In [7]:
target_index = src.loc[src.targetlabel ==1].index
print("number of targeted source tweets: ",len(target_index))
target_index

number of targeted source tweets:  84


Int64Index([  0,   1,   2,   4,   6,   7,   9,  10,  12,  13,  16,  17,  18,
             20,  22,  24,  28,  33,  34,  43,  45,  46,  52,  55,  57,  59,
             67,  69,  70,  71,  72,  73,  77,  78,  80,  82,  84,  85,  88,
             89,  91,  93,  99, 100, 102, 115, 121, 123, 140, 142, 146, 148,
            155, 174, 177, 187, 190, 195, 196, 207, 208, 213, 222, 224, 226,
            228, 231, 239, 264, 285, 327, 333, 343, 351, 354, 357, 358, 359,
            368, 370, 371, 375, 389, 391],
           dtype='int64')

# Calculate sentiment changes before target

In [8]:
# create "before" dictionary to store sentiments of non-targeting srcs,
# where has three values: ratios of three sentiments/ negative to positive ratio / 
# and indices of those non-targeting srcs
j = 0
before = {"sentiment": [],"np_ratio" :[],"index":[]}
target = list()
for i in target_index:
  if j == i: #the next source tweet also had target label == 1 
    before['sentiment'].append(np.nan)
    before['np_ratio'].append(np.nan)
    before['index'].append(np.nan)
    b = src.loc[i,'src_sentiment']
    target.append(b)
    
  else:     
    count = src.loc[j:i-1,'src_sentiment'].value_counts(normalize=True,sort=False)
    # ratio of three sentiments
    sen = count.round(2).to_dict()
    before['sentiment'].append(sen)
    #ratio of Negative to Positive
    try:
      neg = count['Negative']
      pos = count['Positive']
    except:
      if 'Positive' in count.index:
        # some positive tws but no negative ones
        n_p = 0
      elif "Negative" in count.index:
        # some negative tws but no positive ones
        n_p = np.inf
      else:
        # only neutral tws
        n_p = 0 
    else:
      n_p = round(neg/pos,2)
    finally:
      before['np_ratio'].append(n_p)
    before['index'].append([*range(j,i)])
    # target's sentiment
    b = src.loc[i,'src_sentiment']
    target.append(b)

  j =i+1

In [9]:
pd.set_option('display.max_colwidth', None)
sentiment_change = pd.DataFrame({"sentiment_beforetarget" : before['sentiment'],
                                 "neg_to_pos_beforetarget" : before['np_ratio'],
                                 "sentiment_target" : target,
                                 "target_index": target_index})


# Calculate sentiment changes after target arrived until the next one came.

In [10]:
# calculte the sentiment changes after the "last" targeing src arrive.
# save it into "last_value" variable.

sentiment_aftertarget = sentiment_change['sentiment_beforetarget']
last_value = src.loc[target_index[-1]+1: ,'src_sentiment'].value_counts(normalize=True,sort=False).round(2).to_dict()
sentiment_aftertarget = pd.concat([sentiment_aftertarget[1:] ,pd.Series([last_value])], ignore_index=True )
sentiment_aftertarget

0                                                       NaN
1                                                       NaN
2                                          {'Neutral': 1.0}
3                                          {'Neutral': 1.0}
4                                                       NaN
                              ...                          
79                                                      NaN
80                      {'Negative': 0.67, 'Neutral': 0.33}
81    {'Positive': 0.31, 'Neutral': 0.38, 'Negative': 0.31}
82                                        {'Negative': 1.0}
83     {'Positive': 0.17, 'Neutral': 0.53, 'Negative': 0.3}
Length: 84, dtype: object

In [11]:
sentiment_change['sentiment_aftertarget'] = sentiment_aftertarget
sentiment_change

,sentiment_beforetarget,neg_to_pos_beforetarget,sentiment_target,target_index,sentiment_aftertarget
0,NaN,NaN,Neutral,0,NaN
1,NaN,NaN,Neutral,1,NaN
2,NaN,NaN,Neutral,2,{'Neutral': 1.0}
3,{'Neutral': 1.0},0.0,Positive,4,{'Neutral': 1.0}
4,{'Neutral': 1.0},0.0,Negative,6,NaN
...,...,...,...,...,...
79,{'Neutral': 1.0},0.0,Negative,370,NaN
80,NaN,NaN,Negative,371,"{'Negative': 0.67, 'Neutral': 0.33}"
81,"{'Negative': 0.67, 'Neutral': 0.33}",inf,Negative,375,"{'Positive': 0.31, 'Neutral': 0.38, 'Negative': 0.31}"
82,"{'Positive': 0.31, 'Neutral': 0.38, 'Negative': 0.31}",1.0,Negative,389,{'Negative': 1.0}


In [12]:
neg = last_value['Negative']
pos = last_value['Positive'] 
n_p = round(neg/pos,2)

neg_to_pos_aftertarget = sentiment_change['neg_to_pos_beforetarget'][1:]
neg_to_pos_aftertarget = pd.concat([neg_to_pos_aftertarget, pd.Series([n_p])], ignore_index=True )

sentiment_change['neg_to_pos_aftertarget'] = neg_to_pos_aftertarget
sentiment_change

,sentiment_beforetarget,neg_to_pos_beforetarget,sentiment_target,target_index,sentiment_aftertarget,neg_to_pos_aftertarget
0,NaN,NaN,Neutral,0,NaN,NaN
1,NaN,NaN,Neutral,1,NaN,NaN
2,NaN,NaN,Neutral,2,{'Neutral': 1.0},0.00
3,{'Neutral': 1.0},0.0,Positive,4,{'Neutral': 1.0},0.00
4,{'Neutral': 1.0},0.0,Negative,6,NaN,NaN
...,...,...,...,...,...,...
79,{'Neutral': 1.0},0.0,Negative,370,NaN,NaN
80,NaN,NaN,Negative,371,"{'Negative': 0.67, 'Neutral': 0.33}",inf
81,"{'Negative': 0.67, 'Neutral': 0.33}",inf,Negative,375,"{'Positive': 0.31, 'Neutral': 0.38, 'Negative': 0.31}",1.00
82,"{'Positive': 0.31, 'Neutral': 0.38, 'Negative': 0.31}",1.0,Negative,389,{'Negative': 1.0},inf


## calculate number of rows

In [13]:
# calculate number of rows before the target sentiment arrived.
row_before = []
for i in range(len(target_index)):
  if i >0 :
    row_before.append(target_index[i] - target_index[i-1]-1)
  else:
    row_before.append(target_index[i])
print(row_before)


[0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 2, 0, 0, 1, 1, 1, 3, 4, 0, 8, 1, 0, 5, 2, 1, 1, 7, 1, 0, 0, 0, 0, 3, 0, 1, 1, 1, 0, 2, 0, 1, 1, 5, 0, 1, 12, 5, 1, 16, 1, 3, 1, 6, 18, 2, 9, 2, 4, 0, 10, 0, 4, 8, 1, 1, 1, 2, 7, 24, 20, 41, 5, 9, 7, 2, 2, 0, 0, 8, 1, 0, 3, 13, 1]


In [14]:
# calculate number of rows after the target sentiment arrived.
row_after = []
for i in range(len(target_index)):
  if i+1 < len(target_index):
    row_after.append(target_index[i+1] - target_index[i]-1)
  else: #last one
    row_after.append(src.index[-1]-target_index[i])
print(row_after)


[0, 0, 1, 1, 0, 1, 0, 1, 0, 2, 0, 0, 1, 1, 1, 3, 4, 0, 8, 1, 0, 5, 2, 1, 1, 7, 1, 0, 0, 0, 0, 3, 0, 1, 1, 1, 0, 2, 0, 1, 1, 5, 0, 1, 12, 5, 1, 16, 1, 3, 1, 6, 18, 2, 9, 2, 4, 0, 10, 0, 4, 8, 1, 1, 1, 2, 7, 24, 20, 41, 5, 9, 7, 2, 2, 0, 0, 8, 1, 0, 3, 13, 1, 107]


In [15]:
sentiment_change['count_of_rows_bef'] = pd.Series(row_before)
sentiment_change['count_of_rows_after'] = pd.Series(row_after)

In [16]:
sentiment_change.columns

Index(['sentiment_beforetarget', 'neg_to_pos_beforetarget', 'sentiment_target',
       'target_index', 'sentiment_aftertarget', 'neg_to_pos_aftertarget',
       'count_of_rows_bef', 'count_of_rows_after'],
      dtype='object')

In [17]:
sentiment_change = sentiment_change[['sentiment_beforetarget', 'neg_to_pos_beforetarget','count_of_rows_bef',  
                                    'sentiment_target',
                                    'sentiment_aftertarget', 'neg_to_pos_aftertarget','count_of_rows_after']]
sentiment_change

,sentiment_beforetarget,neg_to_pos_beforetarget,count_of_rows_bef,sentiment_target,sentiment_aftertarget,neg_to_pos_aftertarget,count_of_rows_after
0,NaN,NaN,0,Neutral,NaN,NaN,0
1,NaN,NaN,0,Neutral,NaN,NaN,0
2,NaN,NaN,0,Neutral,{'Neutral': 1.0},0.00,1
3,{'Neutral': 1.0},0.0,1,Positive,{'Neutral': 1.0},0.00,1
4,{'Neutral': 1.0},0.0,1,Negative,NaN,NaN,0
...,...,...,...,...,...,...,...
79,{'Neutral': 1.0},0.0,1,Negative,NaN,NaN,0
80,NaN,NaN,0,Negative,"{'Negative': 0.67, 'Neutral': 0.33}",inf,3
81,"{'Negative': 0.67, 'Neutral': 0.33}",inf,3,Negative,"{'Positive': 0.31, 'Neutral': 0.38, 'Negative': 0.31}",1.00,13
82,"{'Positive': 0.31, 'Neutral': 0.38, 'Negative': 0.31}",1.0,13,Negative,{'Negative': 1.0},inf,1


# save file

In [18]:
savepath = 'sydneysiege/18 targetanalysis/twanalysis/src_target_sentiment.csv'

with open(savepath, 'w', encoding = 'utf-8') as f:
  sentiment_change.to_csv(f)